# 🎬 AI Short-Video Creator - Google Colab

This notebook runs the AI Short-Video Creator in Google Colab with GPU acceleration.

## 🚀 Setup Instructions:
1. **Enable GPU**: Runtime → Change runtime type → GPU (T4)
2. **Run all cells** in order
3. **Click the ngrok URL** when it appears
4. **Upload your text** and create videos!

---

In [ ]:
pip install TTS


In [18]:
# 🔧 System Setup and Dependencies
print("📦 Installing system dependencies...")
!apt-get update -qq
!apt-get install -y ffmpeg

print("🐍 Installing Python packages...")
!pip install -q streamlit torch torchvision torchaudio transformers tts moviepy opencv-python pillow requests python-dotenv nltk pydub beautifulsoup4 PyPDF2 python-docx pyngrok

print("📚 Downloading NLTK data...")
import nltk
nltk.download('punkt', quiet=True)

print("✅ Setup complete!")

📦 Installing system dependencies...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
🐍 Installing Python packages...
ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0, <3.9; 0.0.10.3 Requires-Python >=3.6.0, <3.9; 0.0.11 Requires-Python >=3.6.0, <3.9; 0.0.12 Requires-Python >=3.6.0, <3.9; 0.0.13.1 Requires-Python >=3.6.0, <3.9; 0.0.13.2 Requires-Python >=3.6.0, <3.9; 0.0.14.1 Requires-Python >=3.6.0, <3.9; 0.0.15 Requires-Python >=3.6.0, <3.9; 0.0.15.1 Requires-Python >=3.6.0, <3.9; 0.0.9 Requires-Python >=3.6.0, <3.9; 0.0.9.1 Requires-Python >=3.6.0, <3.9; 0.0.9.2 

In [19]:
# 🔥 GPU Check
import torch
print(f"🚀 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ GPU not available - will use CPU (slower)")

🚀 GPU Available: True
🎯 GPU Name: Tesla T4
💾 GPU Memory: 15.8 GB


In [ ]:
# 📥 Clone Repository
import os
if os.path.exists('/content/ai-short-video-creator'):
    print("📂 Repository already exists, updating...")
    %cd /content/ai-short-video-creator
    !git pull
else:
    print("📥 Cloning repository...")
    # 📥 Clone Repository
    !git clone https://github.com/Shubham04-oss/pixelauto.git
    %cd pixelauto

print("📁 Current directory:", os.getcwd())
!ls -la

📥 Cloning repository...
Cloning into 'pixelauto'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29 (delta 1), reused 29 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 45.11 KiB | 11.28 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
# ⚙️ Configure for Colab
print("⚙️ Configuring for Colab...")

# Create Colab-specific config
colab_config = """
# Colab-specific settings
USE_GPU=true
MAX_MEMORY=6
BATCH_SIZE=1
VIDEO_DURATION=45
CACHE_MODELS=true
DEBUG_MODE=false
"""

with open('.env', 'w') as f:
    f.write(colab_config)

print("✅ Configuration complete!")

In [ ]:
pip install pyngrok

In [ ]:
# 🌐 Setup Ngrok (Get free token from ngrok.com)
from pyngrok import ngrok, conf

# Replace with your ngrok token (get free one from ngrok.com)
NGROK_TOKEN = "31YGPNmIqcUAHrenZKh8YiJh8Ds_6ov4E4HeFZJ73KtW3vEqN"  # Paste your token here

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ Ngrok configured!")
else:
    print("⚠️ Please get a free ngrok token from https://ngrok.com and paste it above")
    print("   This is needed to access your app from the internet")

In [ ]:
# 🤖 Download AI Models (First run only)
print("🤖 Downloading AI models... (this may take 5-10 minutes on first run)")

try:
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    print("📥 Downloading BART model...")
    tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
    print("✅ BART model ready!")
except Exception as e:
    print(f"⚠️ BART download failed: {e}")

try:
    from TTS.api import TTS
    print("📥 Downloading TTS model...")
    tts = TTS('tts_models/en/ljspeech/tacotron2-DDC', progress_bar=False)
    print("✅ TTS model ready!")
except Exception as e:
    print(f"⚠️ TTS download failed: {e}")

print("🎉 Model download complete!")

In [ ]:
# 🚀 Launch Application
import subprocess
import threading
import time
from pyngrok import ngrok

print("🚀 Starting AI Short-Video Creator...")

# Function to run Streamlit
def run_streamlit():
    subprocess.run([
        'streamlit', 'run', 'app/main.py',
        '--server.port=8501',
        '--server.address=0.0.0.0',
        '--server.headless=true',
        '--browser.gatherUsageStats=false'
    ])

# Start Streamlit in background
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

# Wait for Streamlit to start
print("⏳ Waiting for Streamlit to start...")
time.sleep(15)

# Create public URL
try:
    public_url = ngrok.connect(port='8501')
    print(f"""🎉 SUCCESS! Your AI Video Creator is running!

🌐 Public URL: {public_url}

📱 Click the link above to access your application!

🎬 Ready to create amazing videos from text!

💡 Tips:
   • Use the sample files in assets/samples/ to test
   • Upload articles 200-800 words for best results
   • Videos are generated in 9:16 format for social media
   • Processing takes 2-5 minutes per video

🔧 Troubleshooting:
   • If link doesn't work, wait 30 seconds and try again
   • Check that GPU is enabled in Runtime settings
   • Restart kernel if you encounter errors
    """)
except Exception as e:
    print(f"❌ Failed to create public URL: {e}")
    print("💡 Make sure you've set your ngrok token above")

In [ ]:
# 📊 Monitor Application (Optional)
import psutil
import time

def monitor_resources():
    while True:
        cpu_percent = psutil.cpu_percent(interval=1)
        memory = psutil.virtual_memory()
        gpu_memory = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0

        print(f"💻 CPU: {cpu_percent:.1f}% | 💾 RAM: {memory.percent:.1f}% | 🔥 GPU: {gpu_memory:.1f}GB")
        time.sleep(30)

# Uncomment to monitor resources (will run continuously)
# monitor_resources()

## 🎯 Usage Instructions

1. **Click the public URL** generated above
2. **Upload a text file** or paste content (200-800 words recommended)
3. **Configure settings** (video length, style, voice)
4. **Click Generate Video** and wait 2-5 minutes
5. **Download your video** in perfect 9:16 format for social media!

## 📱 Perfect for:
- Instagram Reels
- YouTube Shorts
- TikTok videos
- LinkedIn video posts
- Twitter video content

## 🔧 Troubleshooting:
- **App not loading?** Wait 30 seconds, models might still be downloading
- **Out of memory?** Restart kernel and try again
- **Slow processing?** Make sure GPU is enabled in Runtime settings
- **No public URL?** Set your ngrok token in the setup cell

